In [12]:
# import libraries 
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List,Dict
from langchain.agents import AgentExecutor
import json
from langchain_core.agents import AgentActionMessageLog, AgentFinish
from abc import ABC, abstractmethod
from prompts import RESUME_PROMPTS
from inputs import USER_RESUME,JOB_DESCRIPTION

In [13]:
# Setting environment variables 
load_dotenv()

True

In [14]:
class BaseGeneratorLLM(ABC):

    @abstractmethod
    def generate_content(user_resume:str,job_description:str):
        pass

In [15]:
# Generator LLM
class GeneratorLLM(BaseGeneratorLLM):

    def __init__(self,prompt_data:str,model_name:str = "gpt-4-1106-preview",temperature:float = 0.2):
        self.llm = ChatOpenAI(model=model_name, temperature=temperature)
        self.prompt_data : str = prompt_data
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system",f"""You are an experienced resume writer with over 10 years of experience.
                            {prompt_data}"""),
                ("user", "User résumé: {user_resume} \n\n\n\n\n Job Description:{job_description}")
            ]
        )

    def generate_content(self,user_resume :str,job_description:str) -> str:
        self.chain = self.prompt | self.llm
        response : json = self.chain.invoke({"user_resume":user_resume, "job_description":job_description})
        return response.content    
        

In [16]:
class FactoryGeneratorLLM():

    def __init__(self, prompts : str):
        self.prompts = prompts
 
    def generate_object(self,category:str) -> BaseGeneratorLLM :

        if category == "Summary":
            prompt_data : str = self.prompts["Summary"]  
        elif category == "Experience":
            prompt_data : str = self.prompts["Experience"]
        elif category == "Projects":
            prompt_data : str = self.prompts["Projects"]
        else:
            raise Exception ("Category not supported")
        
        llm_agent : BaseGeneratorLLM = GeneratorLLM(prompt_data)
        return llm_agent

In [17]:
class AssessResponse(BaseModel):
    score: int = Field(description = "Score of the resume given based on scale of 1-10")
    feedback: List[str] = Field(description="List of feedback provided to improve the resume.")

In [18]:
class AssessBaseLLM(ABC):
    
    @abstractmethod
    def assess(self,prompt: str):
        pass

In [19]:
class AssessLLM(AssessBaseLLM):
    
    def __init__(self,model_name="gpt-3.5-turbo",temperature=0.0):        
        self.llm = ChatOpenAI(model=model_name, temperature=temperature)
        self.llm_with_tools = self.llm.bind_functions([AssessResponse])
        
    
    def assess(self,user_section: str,job_description: str,section:str) -> AssessResponse:
        self.prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",f"""You are an experienced recruiter writer with over 10 years of experience.
					This is {section} section of the resume rate it from scale of 1-10 and also provide some feedback to imporve the resume if available.
					Critize wherever possible
					""",
                ),
                ("user", "User Section: {user_section} \n\n\n\n\n Job Description:{job_description}"),
            ]
        )

        def parse(output):
            # If no function was invoked, return to user
            if "function_call" not in output.additional_kwargs:
                return AgentFinish(return_values={"output": output.content}, log=output.content)

            # Parse out the function call
            function_call = output.additional_kwargs["function_call"]
            name = function_call["name"]
            inputs = json.loads(function_call["arguments"])

            # If the Response function was invoked, return to the user with the function inputs
            if name == "AssessResponse":
                return AgentFinish(return_values=inputs, log=str(function_call))
            # Otherwise, return an agent action
            else:
                return AgentActionMessageLog(
                    tool=name, tool_input=inputs, log="", message_log=[output]
                )

        self.agent = (
                {
                    "user_section": lambda x: x["user_section"],
                    "job_description": lambda x : x["job_description"],
                }
                | self.prompt
                | self.llm_with_tools
                | parse
        )
        self.agent_executor = AgentExecutor(tools = [],agent=self.agent, verbose=False)
        
        response = self.agent_executor.invoke({"job_description" : job_description,"user_section" :user_section},return_only_outputs=True)
        return AssessResponse(**response)        

In [20]:
class RewriteResume():
    
    def __init__(self,user_resume:Dict,job_description:str,retries: int = 3):
        self.user_resume : Dict = user_resume
        self.job_description : str = job_description
        self.retries : int = retries
        self.factory: FactoryGeneratorLLM = FactoryGeneratorLLM(RESUME_PROMPTS)
        self.assess_llm : AssessLLM = AssessLLM()

    def generate_resume(self):
        
        for key,value in self.user_resume.items():
            print("Generating Resume Section for : ",key)
            # Generate the resume response
            resumer_generator : BaseGeneratorLLM = self.factory.generate_object(key)
            content : str = resumer_generator.generate_content(value,self.job_description)
            
            print("Content : ",content)
            
            # Assess the content 
            score: AssessResponse = self.assess_llm.assess(content,self.job_description,key)
            print("\n\n")
            print("Score : ",score)
            
            print("\n\n\n" + "#" * 100 + "\n\n\n")

In [21]:
rewriteResume = RewriteResume(user_resume=USER_RESUME,job_description=JOB_DESCRIPTION)

In [22]:
rewriteResume.generate_resume()

Generating Resume Section for :  Summary
Content :  **Professional Summary: Software Development Specialist**

As a highly motivated Software Developer with over 1.5 years of experience in the tech industry, I have honed my skills in web development, AI, and software engineering. My expertise lies in Java, Eclipse IDE, and web services within client-server environments, complemented by a robust knowledge of code versioning tools like Git. With a master's degree in computer science, I am adept at problem-solving, machine learning, and algorithm development. My ability to quickly adapt to new technologies and my commitment to continuous learning align with the dynamic nature of modern software development. I am recognized for my strong analytical skills, excellent communication abilities, and a track record of delivering high-quality software solutions. My experience includes providing 24x7 on-call support and participating in project planning, ensuring that I am well-prepared to contrib